# Rescan photos scripts

In [1]:
import time

import firebase_admin
import mpmath
from firebase_admin import credentials
from firebase_admin import firestore

cred = credentials.Certificate(r"C:\!Dev\projects\book_detection\keys\avea-biblosphere-210106-firebase-adminsdk-s3bjc-ff2c6ac275 (1).json")
firebase_admin.initialize_app(cred)

db = firestore.client()

#### Not valid photos

In [7]:
res = db.collection('photos').where(u'valid', u'==', False).stream()
finded = set()
for doc in res:
    finded.add(doc.id)
print('not valid:', len(finded))
#finded

not valid: 95


In [6]:
res = db.collection('photos').where(u'recognition_stats.record_in_stats', u'==', False).stream()
finded = set()
count = 0
for doc in res:
    finded.add(doc.id)
    # update = {'valid': True}
    # db.collection('photos').document(doc.id).update(update)
    # update = {'valid': False}
    # db.collection('photos').document(doc.id).update(update)


print('records not in stats:', len(finded))
#finded

records not in stats: 0


In [5]:
res = db.collection('photos').where(u'status', u'==', 'failed').stream()
finded = set()
for doc in res:
    finded.add(doc.id)
print('status == failed:', len(finded))
#finded

status == failed: 144


#### Photos without block 'recognition_stats'

In [10]:
res = db.collection('photos').stream()
finded = set()
for doc in res:
    data = doc.to_dict()
    if 'valid' in data and not data['valid']:
        continue
    if 'recognition_stats' in data:
        continue
    finded.add(doc.id)

print('finded', len(finded))
#finded

finded 1138


#### Rescan one photo

In [25]:
import requests
photo_id = '3W82C9BUmTxM7D0agG2q'
url = 'https://us-central1-biblosphere-210106.cloudfunctions.net/rescan_photo'

doc = db.collection('photos').document(photo_id).get()
data = doc.to_dict()

res = requests.post(url, json={'photo_id': photo_id})
json = res.json()
print(res.status_code, res.reason)
print('duration:', json['duration'])
print('recognized:', len(json['recognized_books']))
for n, el in enumerate(json['recognized_books'], 1):
    print(f'{n}) {el}')

200 OK
duration: 0:00:26.506920
recognized: 1
1) 9780812039214 Albert Einstein and the Theory of Relativity (Barrons Solution Series) Robert Cwiklik


#### Rescan several photos

In [30]:
import requests
url = 'https://us-central1-biblosphere-210106.cloudfunctions.net/rescan_photo'
batch_size = 2

res = db.collection('photos').stream()
count = 1
for doc in res:
    if count > batch_size:
        break

    data = doc.to_dict()
    if 'valid' in data and not data['valid']:
        continue
    if 'recognition_stats' in data:
        continue

    print(f'{count}) Rescaning photo: {doc.id}')

    if not data.keys() >= {'photo', 'reporter', 'bookplace', 'id', 'location', 'url'}:
        print('ERROR: photo/reporter/bookplace/id/location/url parameters missing.', doc.id, '\n')
        update = {'valid': False}
        db.collection('photos').document(doc.id).update(update)
        continue

    old_recognized = data['recognized'] if 'recognized' in data else 0
    res = requests.post(url, json={'photo_id': doc.id})
    json = res.json()
    print(f"duration: {json['duration']}, recognized: {len(json['recognized_books'])}({old_recognized})\n")

    count += 1

1) Rescaning photo: 2sbdY7e3VD8hIVcoeh1s
duration: 0:00:27.369074, recognized: 0(10)

2) Rescaning photo: 3W82C9BUmTxM7D0agG2q
duration: 0:00:22.247033, recognized: 0(0)



#### Rescan several photos (threads)

In [3]:
import numpy as np
import time

def fill_array(size):
    res = db.collection('photos').stream()
    result = []
    count = 1
    for doc in res:

        if count > batch_size * threads:
            break

        data = doc.to_dict()
        if 'valid' in data and not data['valid']:
            continue
        if 'recognition_stats' in data:
            continue

        if not data.keys() >= {'photo', 'reporter', 'bookplace', 'id', 'location', 'url'}:
            print('ERROR: photo/reporter/bookplace/id/location/url parameters missing.', doc.id, '\n')
            update = {'valid': False}
            db.collection('photos').document(doc.id).update(update)
            continue
        result.append(doc.id)
        count += 1

    return result


def rescan_in_thread(name: str, photo_ids: list):
    url = 'https://us-central1-biblosphere-210106.cloudfunctions.net/rescan_photo'
    for n, id in enumerate(photo_ids):
        print(f'{name}: {n+1}) scanning photo: {id}')
        res = requests.post(url, json={'photo_id': id})
        print(res.status_code)



In [4]:
%%time

import requests
from threading import Thread

batch_size = 3
threads = 2

print('--- preparing photo ids ---')
ids = fill_array(batch_size * threads)
print(f'photos: {len(ids)} \n')

print('---starting threads ---')
threads_list = []
for i in range(threads):
    start_ind = i * batch_size
    end_ind = start_ind + batch_size
    thread = Thread(target=rescan_in_thread, args=(f'thread_{i}', ids[start_ind:end_ind]))
    thread.start()
    threads_list.append(thread)

for thread in threads_list:
    thread.join()

print('---finish----')

--- preparing photo ids ---
ERROR: photo/reporter/bookplace/id/location/url parameters missing. 8Rog0PGHTmCMbBYttBFV 

ERROR: photo/reporter/bookplace/id/location/url parameters missing. 8eb77vVcHm5M4GLNV7yA 

photos: 6 

---starting threads ---
thread_0: 1) scanning photo: 7MpISTN2OS1Q7l0cH1Pa
thread_1: 1) scanning photo: 8SuxFxlnZkN0cq7yeonG
200
thread_0: 2) scanning photo: 7ov4FV2ixBO98jw3RQDT
200
thread_0: 3) scanning photo: 89WCVEWsvMP2CaXoWYGw
200
200
thread_1: 2) scanning photo: 8UGrloIEXDP776knW1sC
200
thread_1: 3) scanning photo: 8ehUapMrlf6uZy4dWWNH
503
---finish----
Wall time: 1min 31s


In [7]:
from PIL import Image
import requests
from io import BytesIO

photo_id = '2sbdY7e3VD8hIVcoeh1s'

doc = db.collection('photos').document(photo_id).get()
data = doc.to_dict()
print(data['url'])
response = requests.get(data['url'])
#response.content
img = Image.open(BytesIO(response.content))
img.show()

https://firebasestorage.googleapis.com/v0/b/biblosphere-210106.appspot.com/o/images%2FPChIJuxcSoobjoI8R1I9kKQU4n9w%2F1611266267833.jpg?alt=media&token=915ab76c-f645-44b5-a28f-6b5f688015fb


In [22]:
photo_id = '2sbdY7e3VD8hIVcoeh1s'
doc = db.collection('photos').document(photo_id).get()
data = doc.to_dict()
data

{'total': 31,
 'location': {'geohash': 'd1u0x26gp2s0',
  'geopoint': <google.cloud.firestore_v1._helpers.GeoPoint at 0x25691990940>},
 'bookplace': 'PChIJuxcSoobjoI8R1I9kKQU4n9w',
 'recognized': 10,
 'reporter': 'Syiru36ehIglQqOk2wNpgpNqG982',
 'photo': 'images/PChIJuxcSoobjoI8R1I9kKQU4n9w/1611266267833.jpg',
 'id': '2sbdY7e3VD8hIVcoeh1s',
 'count': 7,
 'url': 'https://firebasestorage.googleapis.com/v0/b/biblosphere-210106.appspot.com/o/images%2FPChIJuxcSoobjoI8R1I9kKQU4n9w%2F1611266267833.jpg?alt=media&token=915ab76c-f645-44b5-a28f-6b5f688015fb',
 'status': 'failed',
 'thumbnail': 'https://storage.googleapis.com/biblosphere-210106.appspot.com/thumbnails/PChIJuxcSoobjoI8R1I9kKQU4n9w/1611266267833.jpg'}

In [75]:
photo_id = '3s6Am4UxI1EAT8JzHplv'
update = {'valid': False}
db.collection('photos').document(photo_id).update(update)

update_time {
  seconds: 1632503203
  nanos: 298959000
}